In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [7]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model='gpt-4.1',
    instructions='당신은 영화평론가야!',
    input='스타워즈 시리즈의 다스베이더의 역할에 대해 한 줄로 설명해줘'
)
print(response.output_text)

다스 베이더는 스타워즈 시리즈에서 타락한 영웅이자 은하 제국의 암흑 군주로, 은하계의 운명을 뒤흔드는 중심 인물이다.


In [8]:
response = client.responses.create(
    model='gpt-4.1',
    input=[
        {
            "role" : "developer",
            "content" : '음식에 대한 이야기 하는 것을 좋아해'
        },
        {
            "role" : "user",
            "content" : '오늘은 무엇을 먹을까'
        }
    ]
)
print(response.output_text)

정말 좋은 질문이야! 오늘은 기분이 어떤지, 그리고 어떤 음식을 먹고 싶은지에 따라 추천할 메뉴가 달라질 수 있어. 몇 가지 질문에 답해줄 수 있어?

1. 집에 있는 재료로 뭘 만들고 싶어? 아니면 외식을 하고 싶어?
2. 국, 찌개, 볶음, 면, 밥, 샐러드 등 어떤 종류가 땡겨?
3. 매운 음식 좋아해?
4. 가볍게 먹고 싶어, 든든하게 먹고 싶어?

예를 들어,

- **가볍고 건강하게**: 샐러드나 연어덮밥, 비빔국수
- **든든하게**: 김치찌개, 된장찌개, 제육볶음, 치킨
- **특별하게**: 떡볶이, 라멘, 파스타, 초밥
- **외식**: 족발, 피자, 햄버거, 샤브샤브

어떤 쪽이 끌려? 추가로 특정 재료나 선호하는 음식 있으면 알려줘!


In [ ]:
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {
            'role' : 'user',
            'content' : '스타워즈 시리즈에 대해 한 문장으로 설명해줘'
        }
    ]
)

In [16]:
# dict(completion.choices[0])
completion.choices[0].message.content

'스타워즈 시리즈는 먼 미래의 우주를 배경으로 제다이와 시스, 포스의 힘을 둘러싼 선과 악의 대결을 그린 SF 영화 시리즈입니다.'

In [ ]:
custom_input = input('질문을 남겨주세요. ')
print('질문 : ', custom_input)

completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=[
        {
            'role' : 'user',
            'content' : custom_input
        }
    ]
)

print('답변 : ', completion.choices[0].message.content)

질문 :  토끼는 왜 코를 자꾸 움직이나요
답변 :  토끼가 코를 자꾸 움직이는 이유에는 몇 가지 중요한 역할이 있습니다.

1. **냄새를 잘 맡기 위해서**
   - 토끼는 코를 움직이며 주변의 냄새를 더 잘 맡을 수 있습니다. 이 행동을 '코를 씰룩인다'고 표현하기도 하는데, 이렇게 하면 코 구멍(비공)이 넓어졌다 좁아졌다 하면서 공기가 더 많이, 더 다양하게 후각 기관에 들어옵니다. 천적의 냄새나 먹이 냄새 같은 중요한 정보를 모으기 위한 거죠.

2. **의사 표현 및 경계 신호**
   - 토끼는 불안하거나 경계할 때도 코를 빠르게 움직입니다. 평소보다 코가 더 빨리 움직인다면 주변에 신경을 쓰고 있다는 의미일 수 있습니다.

3. **체온 조절**
   - 코 움직임을 통해 호흡 속도를 조절할 수 있기 때문에 어느 정도 체온 조절에도 도움이 됩니다.

요약하면, 토끼가 코를 움직이는 것은 외부 환경을 감지하고, 경계하며, 몸 상태를 조절하는 데 꼭 필요한 행동입니다. 아주 건강한 토끼의 자연스러운 습성이니 걱정하지 않으셔도 됩니다!


In [36]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [37]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius",
    "parameters": {
        "type": "object",
        "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]


In [38]:
input_messages = [{"role": "user", "content": " 오늘 파리 날씨 어때?"}]
response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
    )

dict(response.output[0])

{'arguments': '{"latitude":48.8566,"longitude":2.3522}',
 'call_id': 'call_RGh7uFqi3MEkAsDAImbT9WuD',
 'name': 'get_weather',
 'type': 'function_call',
 'id': 'fc_685bb1ad1ca4819bb8b174a787ed775a0aa11903dcdfba93',
 'status': 'completed'}

In [39]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args['latitude'], args['longitude'])
print(result)

23.2


In [40]:
input_messages.append(tool_call)
input_messages.append({
    'type' : 'function_call_output',
    'call_id' : tool_call.call_id,
    'output' : str(result)
})

# 최종 응답
response2 = client.responses.create(
    model='gpt-4.1',
    input=input_messages,
    tools=tools
)

print(response2.output_text)

오늘 파리의 현재 기온은 약 23.2°C입니다. 날씨가 꽤 따뜻하네요! 더 자세한 정보나 다른 도시의 날씨가 궁금하신가요?
